<a href="https://colab.research.google.com/github/tbhogstrom/nylas_rag/blob/main/nylas_openai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: I need to write a notebook that pulls all emails that were sent by a user with the nylas api, then writes each email to a vector database - milvus, then uses that vector database to reply to prompt with the tone of the vector db messages

!pip install nylas milvus pymilvus sentence-transformers

# Import necessary libraries
import nylas
import os
from pymilvus import connections, utility, Collection, FieldSchema, CollectionSchema, DataType
from sentence_transformers import SentenceTransformer
import numpy as np

# Nylas API Credentials
nylas_client_id = "YOUR_NYLAS_CLIENT_ID"
nylas_client_secret = "YOUR_NYLAS_CLIENT_SECRET"
nylas_access_token = "YOUR_NYLAS_ACCESS_TOKEN"  # You can obtain this after user authorization

# Milvus Connection
MILVUS_HOST = "localhost"  # Replace with your Milvus host if it's not local
MILVUS_PORT = "19530"  # Replace with your Milvus port if it's different

# Initialize Nylas Client
nylas_client = nylas.Client(
    client_id=nylas_client_id,
    client_secret=nylas_client_secret,
    access_token=nylas_access_token,
)

# Initialize Sentence Transformer
model = SentenceTransformer('all-MiniLM-L6-v2')


def create_milvus_collection():
    """Creates a Milvus collection for storing emails."""
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if utility.has_collection("email_collection"):
        utility.drop_collection("email_collection")

    fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="email_content", dtype=DataType.VARCHAR, max_length=10000),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    ]
    schema = CollectionSchema(fields=fields, description="Email Collection")
    collection = Collection(name="email_collection", schema=schema)
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "L2",
        "params": {"nlist": 1024},
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection


def embed_email(email_content):
    """Embeds the email content into a vector using Sentence Transformer."""
    return model.encode(email_content).tolist()


def insert_emails_to_milvus(collection, emails):
    """Inserts emails into the Milvus collection."""
    data = []
    for email in emails:
        if email.body:
            email_content = email.body
            embedding = embed_email(email_content)
            data.append([email_content, embedding])
    if data:
        collection.insert([data])


def search_similar_emails(collection, query, top_k=5):
    """Searches for similar emails in Milvus based on a query."""
    query_embedding = model.encode(query).tolist()
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        expr=None,
    )
    return results


# Main Execution
if __name__ == "__main__":
    collection = create_milvus_collection()

    # Fetch emails from Nylas
    emails = nylas_client.emails.list(limit=100)  # Adjust limit as needed

    # Insert emails into Milvus
    insert_emails_to_milvus(collection, emails)

    # Example: Search for similar emails and reply to a prompt
    query = "What is your feedback on the product?"
    results = search_similar_emails(collection, query)

    # Process the results to generate a reply (using the tone and style of retrieved emails)
    reply = "Based on previous emails, here's a potential reply:\n"
    for result in results[0]:
        reply += result.entity.email_content + "\n"
    print(reply)


Here’s a complete script that outlines how you can use the Nylas API to fetch emails sent by a user, store them in a vector database (Milvus), and then use OpenAI to query that database to generate a response in the tone of the user’s emails.

Steps in the Code:

	1.	Nylas API Setup: Fetch emails sent by a specific user.
	2.	Milvus Setup: Create a Milvus collection to store email content and their vector embeddings.
	3.	Embedding Email Content: Use SentenceTransformer to convert email content into vector embeddings.
	4.	Search in Milvus: Query the Milvus vector database to find emails with similar context.
	5.	Generate a Response: Use OpenAI’s API to generate a reply based on the context retrieved from the vector database.

This script can help automate the process of generating email responses in a personalized tone, combining the power of Nylas API, Milvus for vector search, and OpenAI for text generation.

Let me know if you need further customizations!

In [ ]:
# prompt: I want to edit that so that it pulls all emails that were sent by a user with the nylas api, then writes each email to a vector database - milvus, then uses openai to query that vector database to reply to prompt with the tone of the vector db messages

import nylas
import os
from pymilvus import connections, utility, Collection, FieldSchema, CollectionSchema, DataType
from sentence_transformers import SentenceTransformer
import numpy as np
import openai

!pip install nylas milvus pymilvus sentence-transformers openai

# Import necessary libraries

# Nylas API Credentials
nylas_client_id = "YOUR_NYLAS_CLIENT_ID"
nylas_client_secret = "YOUR_NYLAS_CLIENT_SECRET"
nylas_access_token = "YOUR_NYLAS_ACCESS_TOKEN"  # You can obtain this after user authorization

# Milvus Connection
MILVUS_HOST = "localhost"  # Replace with your Milvus host if it's not local
MILVUS_PORT = "19530"  # Replace with your Milvus port if it's different

# OpenAI API Key
openai.api_key = "YOUR_OPENAI_API_KEY"

# Initialize Nylas Client
nylas_client = nylas.Client(
    client_id=nylas_client_id,
    client_secret=nylas_client_secret,
    access_token=nylas_access_token,
)

# Initialize Sentence Transformer
model = SentenceTransformer('all-MiniLM-L6-v2')


def create_milvus_collection():
    """Creates a Milvus collection for storing emails."""
    connections.connect(host=MILVUS_HOST, port=MILVUS_PORT)
    if utility.has_collection("email_collection"):
        utility.drop_collection("email_collection")

    fields = [
        FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=True),
        FieldSchema(name="email_content", dtype=DataType.VARCHAR, max_length=10000),
        FieldSchema(name="embedding", dtype=DataType.FLOAT_VECTOR, dim=384),
    ]
    schema = CollectionSchema(fields=fields, description="Email Collection")
    collection = Collection(name="email_collection", schema=schema)
    index_params = {
        "index_type": "IVF_FLAT",
        "metric_type": "L2",
        "params": {"nlist": 1024},
    }
    collection.create_index(field_name="embedding", index_params=index_params)
    return collection


def embed_email(email_content):
    """Embeds the email content into a vector using Sentence Transformer."""
    return model.encode(email_content).tolist()


def insert_emails_to_milvus(collection, emails):
    """Inserts emails into the Milvus collection."""
    data = []
    for email in emails:
        if email.body and email.sender.email == "your_email@example.com":  # Filter by sender
            email_content = email.body
            embedding = embed_email(email_content)
            data.append([email_content, embedding])
    if data:
        collection.insert([data])


def search_similar_emails(collection, query, top_k=5):
    """Searches for similar emails in Milvus based on a query."""
    query_embedding = model.encode(query).tolist()
    search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
    results = collection.search(
        data=[query_embedding],
        anns_field="embedding",
        param=search_params,
        limit=top_k,
        expr=None,
    )
    return results


def generate_reply_with_openai(query, context):
    """Generates a reply using OpenAI based on the query and context."""
    prompt = f"Based on the following email context, respond to the user's query:\n\nContext:\n{context}\n\nQuery:\n{query}\n\nReply:"
    response = openai.Completion.create(
        engine="text-davinci-003",  # Choose your OpenAI engine
        prompt=prompt,
        max_tokens=150,  # Adjust as needed
        n=1,
        stop=None,
        temperature=0.7,
    )
    return response.choices[0].text.strip()


# Main Execution
if __name__ == "__main__":
    collection = create_milvus_collection()

    # Fetch emails from Nylas
    emails = nylas_client.emails.list(limit=100)  # Adjust limit as needed

    # Insert emails into Milvus
    insert_emails_to_milvus(collection, emails)

    # Example: Search for similar emails and generate a reply using OpenAI
    query = "What is your feedback on the product?"
    results = search_similar_emails(collection, query)

    # Prepare context from retrieved emails
    context = ""
    for result in results[0]:
        context += result.entity.email_content + "\n"

    # Generate reply using OpenAI
    reply = generate_reply_with_openai(query, context)
    print(f"Generated Reply:\n{reply}")


SyntaxError: invalid syntax (<ipython-input-1-405f1a10943a>, line 1)